In [ ]:
import pandas as pd
import pickle


In [ ]:
# support print functions
def print_dict(dict, df_print=False):
    """
    Print the contents of a dictionary with DataFrames in a readable format.
    Input:
        List of dict items.
        df_print: if True, prints DataFrame contents; if False, only print the shape of the DF keys

    Output:
        Prints the keys and values of each dictionary item.
    """
    import pandas as pd
    print(f"\n dict: ({len(dict)} items):")
    for i, d in enumerate(dict):
        print(f"\nItem {i}:")
        print(f"  Keys: {list(d.keys())}")

        for k, v in d.items():
            if isinstance(v, pd.DataFrame):
                print(f"  {k}: <DataFrame shape={v.shape}>")
                if df_print == True: print(f"  {k}: {v}")
            else:
                print(f" {k}: {v}")


In [ ]:
# paths
rt = "/host/verges/tank/data/daniel/3T7T/z/outputs/results"
winStdy_pth = "comparisons_withinStudy_09Jun2025-1127.pkl"
btwStdy_pth = "comp_btwScanners_09Jun2025-1139.pkl"


In [ ]:
# import within study comparisons (winStudy)
# import between study comparisons (btwStudy)
with open(f"{rt}/{winStdy_pth}", "rb") as f:
    winStdy = pickle.load(f)

with open(f"{rt}/{btwStdy_pth}", "rb") as f:
    btwStdy = pickle.load(f)

len(winStdy), len(btwStdy)

In [ ]:
# visualization functions

def showBrains(lh, rh, surface='fsLR-5k', ipsiTo=None, title=None, min=-2.5, max=2.5, inflated=False, save_name=None, save_pth=None, cmap="seismic"):
    """
    Returns brain figures

    inputs:
        lh: column with values for left hemisphere surface (each row represents a vertex and is properly indexed) 
        rh: column right hemisphere surface (each row represents a vertex and is properly indexed) 
        surface: surface type (default is 'fsLR-5k')
        inflated: whether to use inflated surfaces (default is False)
        title: title for the plot (default is None)
        save_name: name to save the figure (default is None)
        save_pth: path to save the figure (default is None)

    """    
    
    import os
    import glob
    import numpy as np
    import nibabel as nib
    import seaborn as sns
    from brainspace.plotting import plot_hemispheres
    from brainspace.mesh.mesh_io import read_surface
    from brainspace.datasets import load_conte69
    import datetime

    micapipe=os.popen("echo $MICAPIPE").read()[:-1]
    
    # set wd to save_pth
    if save_pth is not None:
        if not os.path.exists(save_pth):
            os.makedirs(save_pth)
        os.chdir(save_pth)

    if surface == 'fsLR-5k':
        if inflated == True:
            # Load fsLR 5k inflated
            surf_lh = read_surface(micapipe + '/surfaces/fsLR-5k.L.inflated.surf.gii', itype='gii')
            surf_rh = read_surface(micapipe + '/surfaces/fsLR-5k.R.inflated.surf.gii', itype='gii')
        else:
            # Load Load fsLR 5k
            surf_lh = read_surface(micapipe + '/surfaces/fsLR-5k.L.surf.gii', itype='gii')
            surf_rh = read_surface(micapipe + '/surfaces/fsLR-5k.R.surf.gii', itype='gii')
    elif surface == 'fsLR-32k':
        if inflated == True:
            # Load fsLR 32k inflated
            surf_lh = read_surface(micapipe + '/surfaces/fsLR-32k.L.inflated.surf.gii', itype='gii')
            surf_rh = read_surface(micapipe + '/surfaces/fsLR-32k.R.inflated.surf.gii', itype='gii')
        else:
            # Load fsLR 32k
            surf_lh = read_surface(micapipe + '/surfaces/fsLR-32k.L.surf.gii', itype='gii')
            surf_rh = read_surface(micapipe + '/surfaces/fsLR-32k.R.surf.gii', itype='gii')
    else:
        raise ValueError(f"Surface {surface} not recognized. Use 'fsLR-5k' or 'fsLR-32k'.")

    
    data = np.hstack(np.concatenate([lh, rh], axis=0))


    lbl_text = {'top': title}
    
    if ipsiTo is not None and ipsiTo == "L":
        lbl_text = {
            'left': 'ipsi',
            'right': 'contra'
        }    
    elif ipsiTo is not None and ipsiTo == "R":
        lbl_text = {
            'left': 'contra',
            'right': 'ipsi'
        }
    else:
        lbl_text = {
            'left': 'L',
            'right': 'R'
        }

    # Ensure all values are strings (robust against accidental lists/arrays)
    lbl_text = {k: str(v) for k, v in lbl_text.items()}

    date = datetime.datetime.now().strftime("%d%b%Y-%H%M")
    filename = f"{save_name}_{surface}_{date}.png" if save_name and save_pth else None
    
    # Plot the surface with a title
    if filename : 
        print(f"[showBrains] Plot saved to {filename}")
        return plot_hemispheres(
                surf_lh, surf_rh, array_name=data, 
                size=(900, 250), color_bar='bottom', zoom=1.25, embed_nb=True, interactive=False, share='both',
                nan_color=(0, 0, 0, 1), color_range=(min,max), cmap=cmap, transparent_bg=False, 
                screenshot=True, filename=filename,
                #, label_text = lbl_text
            )
    else:
        return plot_hemispheres(
                surf_lh, surf_rh, array_name=data, 
                size=(900, 250), color_bar='bottom', zoom=1.25, embed_nb=True, interactive=False, share='both',
                nan_color=(0, 0, 0, 1), color_range=(min,max), cmap=cmap, transparent_bg=False, 
                #, label_text = lbl_text
            )


In [ ]:
# for each group, create a column of figures
# top: 7T brains, bottom 3T brains, last comparison
def fig_2():
    """
    input:
        dict1: dictionary item for first plot
        dict2: dictionary item for second plot
        dict3: dictionary item for third plot
        Each dict item must contain 
    output:
        Figure with 3 subplots, each showing a brain surface with values from the respective dictionary item.
    """
    
    showbrains()

In [ ]:
grps = ['allPX', 'TLE_L', 'TLE_R', 'TLE_ic']
fts = ['ADC', 'FA', 'T1map']
surfs = ['fsLR-5k']
lbls = ['midthickness']

for grp in grps:
    for ft in fts:
        for surf in surfs:
            for lbl in lbls:
                print(f"{grp}, {ft}, {surf}, {lbl}")
                # search winStdy and btwStdy for appropriate index
                tT_idx = [i for i, d in enumerate(winStdy) if d['study'] == 'MICs' and d['grp'] == grp and d['feature'] == ft and d['surface'] == surf and d['label'] == lbl]
                sT_idx = [i for i, d in enumerate(btwStdy) if d['study'] == 'PNI' and d['grp'] == grp and d['feature'] == ft and d['surface'] == surf and d['label'] == lbl]
                
                comp_idx = [i for i, d in enumerate(btwStdy) if d['grp'] == grp and d['feature'] == ft and d['surface'] == surf and d['label'] == lbl]
                print(f"\tindices. winStudy [3T: {tT_idx}, 7T: {sT_idx}], comparison: {comp_idx}")
                if not tT_idx or not sT_idx or not comp_idx:
                    print(f"\t\tAt least one comparison incomplete. Skipping.")
                    continue
                
                # Show the brains
                